# Liputan 6
## Klasifikasi jenis berita berdasarkan judul

In [1]:
# install libarary
! pip install pandas
! pip install numpy
! pip install nltk
! pip install Sastrawi
! pip install sklearn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [73]:
# import library

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
# nltk.download()

In [3]:
# load data
df = pd.read_csv("raw.csv")
df.head()

,Judul,Jenis,User Aktif
0,Wajah Istri Sultan Kelantan yang Pernah Bersen...,Lifestyle,7
1,Nonton Video Porno Bareng Pasangan Beri Dampak...,Health,7
2,Cuek dengan Isu Perselingkuhan Behati Prinsloo...,Health,5
3,Skrining Hormon Tiroid pada Bayi Baru Lahir Bi...,Health,5
4,Dibayangi Resesi Harga Emas Bakal Melambung di...,Bisnis,5


In [4]:
# memilih kolom-kolom yang akan di drop 

to_drop = ['User Aktif']

# melakukan drop pada data yang dipilih

df.drop(to_drop, inplace=True, axis=1)

# Hasil penghilenang kolom
df.head()

,Judul,Jenis
0,Wajah Istri Sultan Kelantan yang Pernah Bersen...,Lifestyle
1,Nonton Video Porno Bareng Pasangan Beri Dampak...,Health
2,Cuek dengan Isu Perselingkuhan Behati Prinsloo...,Health
3,Skrining Hormon Tiroid pada Bayi Baru Lahir Bi...,Health
4,Dibayangi Resesi Harga Emas Bakal Melambung di...,Bisnis


In [5]:
# melihat statistik dasar 

df.describe()

,Judul,Jenis
count,800,800
unique,800,4
top,Wajah Istri Sultan Kelantan yang Pernah Bersen...,Lifestyle
freq,1,200


In [6]:
# menghilangkan data judul yang sama
df.drop_duplicates(subset="Judul", keep=False, inplace=True)
df.describe()   

,Judul,Jenis
count,800,800
unique,800,4
top,Wajah Istri Sultan Kelantan yang Pernah Bersen...,Lifestyle
freq,1,200


# Case folding

In [7]:
# mengubah ke lower case
df['Judul'] = df['Judul'].str.lower()
df['Jenis'] = df['Jenis'].str.lower()

df.head()

,Judul,Jenis
0,wajah istri sultan kelantan yang pernah bersen...,lifestyle
1,nonton video porno bareng pasangan beri dampak...,health
2,cuek dengan isu perselingkuhan behati prinsloo...,health
3,skrining hormon tiroid pada bayi baru lahir bi...,health
4,dibayangi resesi harga emas bakal melambung di...,bisnis


In [8]:
# menghilangkan karakter special dan numerik
df['Judul'] = df['Judul'].str.replace('\d*', '')
df['Judul'] = df['Judul'].str.replace('[^\w\s]', ' ')
df['Judul'] = df['Judul'].str.strip()
df.head()

C:\Users\AKU\AppData\Local\Temp\ipykernel_11660\2487329922.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Judul'] = df['Judul'].str.replace('\d*', '')
C:\Users\AKU\AppData\Local\Temp\ipykernel_11660\2487329922.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Judul'] = df['Judul'].str.replace('[^\w\s]', ' ')


,Judul,Jenis
0,wajah istri sultan kelantan yang pernah bersen...,lifestyle
1,nonton video porno bareng pasangan beri dampak...,health
2,cuek dengan isu perselingkuhan behati prinsloo...,health
3,skrining hormon tiroid pada bayi baru lahir bi...,health
4,dibayangi resesi harga emas bakal melambung di,bisnis


# Tokenizing and Stemming

In [9]:
# stemming and tokenizing function to sastrawati
def funSastrawati(val):
    stop_factory = ArrayDictionary(StopWordRemoverFactory().get_stop_words())
    str = StopWordRemover(stop_factory)
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    word = stemmer.stem(str.remove(val))
    return word
# Apply stemming and tokenizing
df['Judul'] = df['Judul'].apply(funSastrawati)
df.head()

,Judul,Jenis
0,wajah istri sultan lantan pernah sengketa miss...,lifestyle
1,nonton video porno bareng pasang beri dampak p...,health
2,cuek isu selingkuh behati prinsloo tangkap kam...,health
3,skrining hormon tiroid bayi baru lahir cegah g...,health
4,bayang resesi harga emas bakal lambung,bisnis


In [10]:
# menyimpan data clean
df.to_csv(r'P:\UKDC\Kapital Selekta\program\clean.csv', index=False, header=True)

In [11]:
# Mengetahui jumlah jenis berita
df['Jenis'].value_counts()

lifestyle    200
health       200
bisnis       200
global       200
Name: Jenis, dtype: int64

In [12]:
df['Label'] = df['Jenis'].map({
    'bisnis' : 0,
    'health' : 1,
    'global' : 2,
    'lifestyle' : 3,
})
df.head()

,Judul,Jenis,Label
0,wajah istri sultan lantan pernah sengketa miss...,lifestyle,3
1,nonton video porno bareng pasang beri dampak p...,health,1
2,cuek isu selingkuh behati prinsloo tangkap kam...,health,1
3,skrining hormon tiroid bayi baru lahir cegah g...,health,1
4,bayang resesi harga emas bakal lambung,bisnis,0


In [13]:
# Spliting data (pemodelan)

X_train, X_test, y_train, y_test = train_test_split(df['Judul'].values, df['Label'].values, test_size=0.1, random_state = 90, stratify=df['Label'].values)

# TF-IDF

In [24]:
# tf-idf
tf = TfidfVectorizer(min_df=1, ngram_range=(1, 3))
tfidf_train = tf.fit_transform(X_train)
tfidf_test = tf.transform(X_test)
print(tfidf_test[:5])

  (0, 7354)	0.46069328407484433
  (0, 6259)	0.46069328407484433
  (0, 5945)	0.433572373675853
  (0, 3855)	0.46069328407484433
  (0, 3222)	0.29175134829630023
  (0, 410)	0.3003019173721372
  (1, 8017)	0.4255724296536098
  (1, 6002)	0.4731939845392052
  (1, 5999)	0.44533716070509755
  (1, 5799)	0.44533716070509755
  (1, 93)	0.44533716070509755
  (2, 4733)	0.35430672334683594
  (2, 3997)	0.48376231528575614
  (2, 3378)	0.43203181605571594
  (2, 2089)	0.3385364436518058
  (2, 1748)	0.5140227174789143
  (2, 1709)	0.2737937184293555
  (3, 8736)	0.5068918316201293
  (3, 6791)	0.4104408571372853
  (3, 5326)	0.5068918316201293
  (3, 603)	0.5636130275871839
  (4, 7823)	0.32640495585139967
  (4, 6680)	0.2320180908227043
  (4, 4057)	0.34926703423275113
  (4, 4056)	0.3366851245729003
  (4, 1943)	0.2437257996678953
  (4, 1500)	0.36548793698808196
  (4, 1479)	0.2506387099069259
  (4, 641)	0.36548793698808196
  (4, 418)	0.3883500153694334
  (4, 410)	0.2531451146745897


# SGD

In [87]:
modelSGD = SGDClassifier(max_iter=1000, tol=1e-3)
modelSGD.fit(tfidf_train,y_train)

SGDClassifier()

In [88]:
y_pred = modelSGD.predict(tfidf_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71        20
           1       0.62      0.75      0.68        20
           2       0.59      0.50      0.54        20
           3       0.79      0.55      0.65        20

    accuracy                           0.65        80
   macro avg       0.66      0.65      0.65        80
weighted avg       0.66      0.65      0.65        80



# Test Sample

In [19]:
test = pd.read_csv("train.csv")
test.head()

,Judul
0,Berikut adalah potret para pemenang yang berha...
1,Selain Sandrinna Michelle masih ada sembilan p...
2,Berikut ini daftar pemenang SCTV Awards 2022 l...
3,Berikutini Link Live Streaming SCTV Awards 202...
4,SCTV award 2022 menganugerahkan aktor Cinta Br...


In [20]:
# menghilangkan karakter special dan numerik
test['Judul'] = test['Judul'].str.replace('\d*', '')
test['Judul'] = test['Judul'].str.replace('[^\w\s]', ' ')
test['Judul'] = test['Judul'].str.strip()

# Apply stemming and tokenizing
test['Judul'] = test['Judul'].apply(funSastrawati)
test.head()

C:\Users\AKU\AppData\Local\Temp\ipykernel_11660\1742386158.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['Judul'] = test['Judul'].str.replace('\d*', '')
C:\Users\AKU\AppData\Local\Temp\ipykernel_11660\1742386158.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test['Judul'] = test['Judul'].str.replace('[^\w\s]', ' ')


,Judul
0,ikut potret menang hasil abadi tim kapanlagi c...
1,selain sandrinna michelle ada sembilan menang ...
2,ikut daftar menang sctv awards lengkap kategor...
3,berikutini link live streaming sctv awards tay...
4,sctv award anugerah aktor cinta brian aktor ut...


In [27]:
# prediksi berita
prediksi = modelSGD.predict(tf.transform(test['Judul'].values))
test['Jenis'] = prediksi
test['Jenis'] = test['Jenis'].map({
    0 : 'bisnis',
    1 : 'health',
    2 : 'global',
    3 : 'lifestyle',
})
test

,Judul,Jenis
0,ikut potret menang hasil abadi tim kapanlagi c...,bisnis
1,selain sandrinna michelle ada sembilan menang ...,lifestyle
2,ikut daftar menang sctv awards lengkap kategor...,bisnis
3,berikutini link live streaming sctv awards tay...,bisnis
4,sctv award anugerah aktor cinta brian aktor ut...,global
...,...,...
126,dramatis marseille curi poin kandang as monaco...,global
127,sead kolasinac buat marseille curi tiga poin k...,bisnis
128,jakarta gol menit akhir sead kolasinac bawa ma...,global
129,marseille hadap as monaco tanding ligue stade ...,global


In [22]:
test.to_csv(r'P:\UKDC\Kapital Selekta\program\result.csv', index=False, header=True)